In [124]:
%matplotlib widget
from data import CBData
can = CBData.loadData('data/candles_2021-01-01T0:00:00__2021-01-31T18:00:00')
CBData.cleanData(can)
can = can.drop_duplicates()
can

,low,high,open,close,volume,del
time,,,,,,
1609459260,29026.97,29083.75,29030.81,29079.56,27.489035,0
1609459320,29027.14,29079.56,29079.56,29053.76,32.481299,60
1609459380,29027.71,29059.40,29051.13,29041.72,8.934692,120
1609459440,29028.52,29052.35,29038.45,29034.30,20.515934,180
1609459500,28983.79,29031.71,29029.75,28997.34,14.545323,240
...,...,...,...,...,...,...
1612115760,32252.71,32382.77,32382.77,32291.58,84.612600,2656500
1612115820,32280.00,32487.46,32291.58,32400.48,43.410869,2656560
1612115880,32215.00,32415.77,32400.08,32223.85,94.305981,2656620


In [125]:
import numpy as np
from filterpy.gh import GHFilter

hyperParam = {}

for w in np.arange(0., .55, .02):
    print(w)
    g = 2 * w
    h = 2 * w**2
    #k = w**3
    hyperParam[str(w)] = {'g': g, 'h':h}
    #import ipdb; ipdb.set_trace()
    f = GHFilter(x = can.iloc[0]['close'], dx = 0., 
                  dt = 1., g = hyperParam[str(w)]['g'], h = hyperParam[str(w)]['h'])
    results = f.batch_filter(list(can['close']))
    #import ipdb; ipdb.set_trace()
    can[f'w:{w:.2f}_x'] = results[1:,0]
    can[f'w:{w:.2f}_dx'] = results[1:,1]
    print(w)

0.0
0.0
0.02
0.02
0.04
0.04
0.06
0.06
0.08
0.08
0.1
0.1
0.12
0.12
0.14
0.14
0.16
0.16
0.18
0.18
0.2
0.2
0.22
0.22
0.24
0.24
0.26
0.26
0.28
0.28
0.3
0.3
0.32
0.32
0.34
0.34
0.36
0.36
0.38
0.38
0.4
0.4
0.42
0.42
0.44
0.44
0.46
0.46
0.48
0.48
0.5
0.5
0.52
0.52
0.54
0.54


In [126]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#plt.ioff()
can.iloc[0:10]
can['delMin'] = can['del']/60
can['delMin'] = pd.to_numeric(can['delMin'], downcast='integer')
can.head()
can.plot(x= 'delMin', y = ['close', 'w:0.02_x', 'w:0.04_x', 'w:0.06_x'])
#can.plot(x= 'delMin', y = ['close', 'w:0.05_x', 'w:0.10_x', 'w:0.15_x'])


#plt.close("all")
#plt.plot(s)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [127]:
import sys
sys.path.append('../../Kalman-and-Bayesian-Filters-in-Python/kf_book')
from book_plots import FloatSlider
from ipywidgets import interact
import matplotlib.pyplot as plt
from filterpy.gh import GHKFilter

#zs1 = gen_data(x0=5, dx=5., count=100, noise_factor=50)
#ddx = -5
#zACC = gen_data(x0=5, dx= 5., ddx = -.01, count=1000, noise_factor=50)
#print(zACC)

raw = can['close'][:5000]

fig = None
def interactive_gh(x, dx, g, h):
    global fig
    if fig is not None: plt.close(fig)
    fig = plt.figure()
    f = GHFilter(x=x, dx=dx, dt=1., g=g, h=h,)
    data = f.batch_filter(raw)
    #print(data[:, 0])
    plt.scatter(range(len(raw)), raw, edgecolor='k', 
                facecolors='none', marker='o', lw=1)
    plt.plot(data[:, 0], color='b')

print(raw.iloc[0])
interact(interactive_gh,           
         x=FloatSlider(value=raw.iloc[0], min=33000, max=42000), 
         dx=FloatSlider(value=0., min=-50, max=50), 
         #ddx=FloatSlider(value=-.1, min=-1, max=1), 
         g=FloatSlider(value=.01, min=.0, max=.02, step=.002, readout_format = '.4f'),
         h=FloatSlider(value=.0001, min=.0, max=5e-4, step=1.e-5, readout_format = '.5f'))
         #k=FloatSlider(value=.00, min=.0, max=.1, step=.0001, readout_format = '.4f'));

29079.56


interactive(children=(FloatSlider(value=33000.0, continuous_update=False, description='x', max=42000.0, min=33…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<function __main__.interactive_gh(x, dx, g, h)>

In [128]:
import sys
sys.path.append('../../Kalman-and-Bayesian-Filters-in-Python/kf_book')
from book_plots import FloatSlider

from ipywidgets import interact
import matplotlib.pyplot as plt
from filterpy.gh import GHKFilter
#zs1 = gen_data(x0=5, dx=5., count=100, noise_factor=50)
#ddx = -5
#zACC = gen_data(x0=5, dx= 5., ddx = -.01, count=1000, noise_factor=50)
#print(zACC)
raw = can['close']

fig = None
def interactive_ghk(x, dx, ddx, g, h, k):
    global fig
    if fig is not None: plt.close(fig)
    fig = plt.figure()
    f = GHKFilter(x=x, dx=dx, dt=1., ddx = ddx, g=g, h=h, k=k)
    data = f.batch_filter(raw)
    #print(data[:, 0])
    days = np.linspace(0, 30, len(raw), endpoint=True)
    plt.scatter(days, raw, edgecolor='k',
                facecolors='none', marker='o', lw=1)
    plt.plot(days, data[1:, 0], color='b')

print(raw.iloc[0])
interact(interactive_ghk,           
         x=FloatSlider(value=raw.iloc[0], min=33000, max=42000), 
         dx=FloatSlider(value=0., min=-50, max=50), 
         ddx=FloatSlider(value=0., min=-1, max=1), 
         g=FloatSlider(value=0.002, min=.0, max=.02, step=.002, readout_format = '.4f'),
         h=FloatSlider(value=1.0e-06, min=.0, max=5e-4, step=1.e-5, readout_format = '.5f'),
         k=FloatSlider(value= 0, min=.0, max=.1, step=.0001, readout_format = '.4f'));

29079.56


interactive(children=(FloatSlider(value=33000.0, continuous_update=False, description='x', max=42000.0, min=33…

In [129]:
#close open figures
plt.close('all')

In [130]:
#some interesting filter values
from filterpy.gh import GHFilter, least_squares_parameters

lsf = GHFilter (0, 0, 1, 0, 0)
z = 10
for i in range(10):
    g,h = least_squares_parameters(i)
    lsf.update(z, g, h)
    print(g, h)

1.0 3.0
1.0 1.0
0.8333333333333334 0.5
0.7 0.3
0.6 0.2
0.5238095238095238 0.14285714285714285
0.4642857142857143 0.10714285714285714
0.4166666666666667 0.08333333333333333
0.37777777777777777 0.06666666666666667
0.34545454545454546 0.05454545454545454


In [131]:
from filterpy.gh import *
g,h,k = optimal_noise_smoothing(.001)
print(g,h,k)
#f = GHKFilter(0,0,0,1,g,h,k)
#f.update(1.)

0.001 5.002501563594566e-07 6.256255376869177e-11


In [132]:
from filterpy.gh.gh_filter import *
critical_damping_parameters(.999)

(0.001998999999999973, 1.0000000000000019e-06)

In [133]:
print(benedict_bornder_constants(.0002))
print(benedict_bornder_constants(.0002, critical=True))

(0.0002, 2.000200020002e-08)
(0.0002, 8.881784197001252e-09)


In [134]:
from ipywidgets import interact
import matplotlib.pyplot as plt
from filterpy.gh import GHKFilter
from filterpy.gh.gh_filter import *
#zs1 = gen_data(x0=5, dx=5., count=100, noise_factor=50)
#ddx = -5
#zACC = gen_data(x0=5, dx= 5., ddx = -.01, count=1000, noise_factor=50)
#print(zACC)
raw = can['close']

fig = None
def benedict_gh(x, dx, g):
    global fig
    if fig is not None: plt.close(fig)
    fig = plt.figure()
    _, h = benedict_bornder_constants(g=g)
    f = GHFilter(x=x, dx=dx, dt=1., g=g, h=h)
    data = f.batch_filter(raw)
    #print(data[:, 0])
    days = np.linspace(0, len(raw)/(24*60), len(raw), endpoint=True)
    plt.scatter(days, raw, edgecolor='k',
                facecolors='none', marker='o', lw=1)
    plt.plot(days, data[1:, 0], color='b')
    print(g, h)


interact(benedict_gh,           
         x=FloatSlider(value=raw.iloc[0], min=25000, max=30000), 
         dx=FloatSlider(value=0., min=-50, max=50), 
         ddx=FloatSlider(value=0., min=-1, max=1), 
         g=FloatSlider(value=0.001, min=.0, max=.02, step=.0001, readout_format = '.4f'));
         #h=FloatSlider(value=1.0e-06, min=.0, max=5e-4, step=1.e-5, readout_format = '.5f'),
         #k=FloatSlider(value= 0, min=.0, max=.1, step=.0001, readout_format = '.4f'));

interactive(children=(FloatSlider(value=29079.56, continuous_update=False, description='x', max=30000.0, min=2…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

x
29079.56
dx
0.00
g
0.0014
Figure 20
x=7.851 y=2.736e+04
0.0014 9.806864805363754e-07

#calc 

In [135]:
# setting GHK
import numpy as np
from filterpy.gh import GHFilter

hyperParam = {}
0.0015
for g in np.linspace(0., 5.e-3, 51):
    print(g)
    _, h = benedict_bornder_constants(g=g)

    hyperParam[str(g)] = {'g': g, 'h':h}
    #import ipdb; ipdb.set_trace()
    f = GHFilter(x = can.iloc[0]['close'], dx = 0., 
                  dt = 1., g = hyperParam[str(g)]['g'], h = hyperParam[str(g)]['h'])
    results = f.batch_filter(list(can['close']))
    #import ipdb; ipdb.set_trace()
    can[f'g:{g:.4f}_x'] = results[1:,0]
    can[f'g:{g:.4f}_dx'] = results[1:,1]
    
    
#f = GHKFilter(x, dx, ddx, dt, g, h, k)

#results = f.batch_filter(list(can['close'])

0.0
0.0001
0.0002
0.00030000000000000003
0.0004
0.0005
0.0006000000000000001
0.0007
0.0008
0.0009000000000000001
0.001
0.0011
0.0012000000000000001
0.0013000000000000002
0.0014
0.0015
0.0016
0.0017000000000000001
0.0018000000000000002
0.0019
0.002
0.0021000000000000003
0.0022
0.0023
0.0024000000000000002
0.0025
0.0026000000000000003
0.0027
0.0028
0.0029000000000000002
0.003
0.0031000000000000003
0.0032
0.0033
0.0034000000000000002
0.0035
0.0036000000000000003
0.0037
0.0038
0.0039000000000000003
0.004
0.0041
0.004200000000000001
0.0043
0.0044
0.0045000000000000005
0.0046
0.0047
0.0048000000000000004
0.0049
0.005


In [136]:
from pprint import pprint as pp
pp(list(can.columns))
print(can.columns)
can['g:0.0014_x']

['low',
 'high',
 'open',
 'close',
 'volume',
 'del',
 'w:0.00_x',
 'w:0.00_dx',
 'w:0.02_x',
 'w:0.02_dx',
 'w:0.04_x',
 'w:0.04_dx',
 'w:0.06_x',
 'w:0.06_dx',
 'w:0.08_x',
 'w:0.08_dx',
 'w:0.10_x',
 'w:0.10_dx',
 'w:0.12_x',
 'w:0.12_dx',
 'w:0.14_x',
 'w:0.14_dx',
 'w:0.16_x',
 'w:0.16_dx',
 'w:0.18_x',
 'w:0.18_dx',
 'w:0.20_x',
 'w:0.20_dx',
 'w:0.22_x',
 'w:0.22_dx',
 'w:0.24_x',
 'w:0.24_dx',
 'w:0.26_x',
 'w:0.26_dx',
 'w:0.28_x',
 'w:0.28_dx',
 'w:0.30_x',
 'w:0.30_dx',
 'w:0.32_x',
 'w:0.32_dx',
 'w:0.34_x',
 'w:0.34_dx',
 'w:0.36_x',
 'w:0.36_dx',
 'w:0.38_x',
 'w:0.38_dx',
 'w:0.40_x',
 'w:0.40_dx',
 'w:0.42_x',
 'w:0.42_dx',
 'w:0.44_x',
 'w:0.44_dx',
 'w:0.46_x',
 'w:0.46_dx',
 'w:0.48_x',
 'w:0.48_dx',
 'w:0.50_x',
 'w:0.50_dx',
 'w:0.52_x',
 'w:0.52_dx',
 'w:0.54_x',
 'w:0.54_dx',
 'delMin',
 'g:0.0000_x',
 'g:0.0000_dx',
 'g:0.0001_x',
 'g:0.0001_dx',
 'g:0.0002_x',
 'g:0.0002_dx',
 'g:0.0003_x',
 'g:0.0003_dx',
 'g:0.0004_x',
 'g:0.0004_dx',
 'g:0.0005_x',
 'g:0.00

time
1609459260    29079.560000
1609459320    29079.523880
1609459380    29079.470929
1609459440    29079.407628
1609459500    29079.292627
                  ...     
1612115760    33691.216883
1612115820    33688.981675
1612115880    33686.501051
1612115940    33684.156347
1612116000    33681.656339
Name: g:0.0014_x, Length: 44261, dtype: float64

In [137]:
%matplotlib widget
close = can['close']
days = np.linspace(0, len(close)/(24*60), len(close), endpoint=True)
plt.scatter(days, close, edgecolor='k',
                facecolors='none', marker='o', lw=1)
plt.plot(days, can['g:0.0010_x'])
plt.plot(days, can['g:0.0005_x'])
#plt.plot(days, can['g:0.0008_x'])
#plt.plot(days, can['g:0.0007_x'])
filParams = {'x0':32640, 'dx':0.0, 'dt':1., 'g':.001, 'h':5.00e-7}
#plt.plot(days, can['g:0.0005_x'])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [138]:
%matplotlib widget
close = can['close']
days = np.linspace(0, len(close)/(24*60), len(close), endpoint=True)
plt.scatter(days, close, edgecolor='k',
                facecolors='none', marker='o', lw=1)
plt.plot(days, can['g:0.0010_dx'])
#plt.plot(days, can['g:0.0005_x'])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [139]:
can['g:0.0010_sdx'] = can['g:0.0010_x'].diff()
temp = can['g:0.0010_sdx'].ewm(span=20,min_periods=0,adjust=False,ignore_na=False).mean()
print(temp)

can['SMA_sdx10'] = can['g:0.0010_sdx'].rolling(window=30).mean()

can[['g:0.0010_sdx', 'g:0.0010_x']]
can['SMA_sdx10']

time
1609459260         NaN
1609459320   -0.025800
1609459380   -0.026945
1609459440   -0.028687
1609459500   -0.033780
                ...   
1612115760   -1.532607
1612115820   -1.555209
1612115880   -1.592398
1612115940   -1.616846
1612116000   -1.649576
Name: g:0.0010_sdx, Length: 44261, dtype: float64


time
1609459260         NaN
1609459320         NaN
1609459380         NaN
1609459440         NaN
1609459500         NaN
                ...   
1612115760   -1.398843
1612115820   -1.414997
1612115880   -1.438133
1612115940   -1.458820
1612116000   -1.482991
Name: SMA_sdx10, Length: 44261, dtype: float64

In [140]:
plt.close('all')
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('time (D)')
ax1.set_ylabel('g:1.e-3', color=color)
ax1.scatter(days, can['close'], color='k')
ax1.plot(days, can['g:0.0010_x'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('dx', color=color)  # we already handled the x-label with ax1

ax2.plot(days, can['SMA_sdx10'], color='tab:pink')
ax2.plot(days, can['g:0.0010_sdx'], color='tab:purple')
ax2.plot(days, can['g:0.0010_dx'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

ax2.axhline(linewidth=2, color='b')
#cushion
cushion = .5
ax2.axhline(y =cushion, linewidth=1, color='b')
ax2.axhline(y =-cushion, linewidth=1, color='b')

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [141]:
ax2.plot(days, can['SMA_sdx10'], color='tab:pink')

In [142]:
print(benedict_bornder_constants(.0002))
print(benedict_bornder_constants(.0002, critical=True))

(0.0002, 2.000200020002e-08)
(0.0002, 8.881784197001252e-09)


In [143]:
print(benedict_bornder_constants(.0002))
print(benedict_bornder_constants(.0002, critical=True))

(0.0002, 2.000200020002e-08)
(0.0002, 8.881784197001252e-09)


In [144]:
import numpy as np
from filterpy.gh import GHFilter
from filterpy.gh.gh_filter import *
from data import CBData

canCrit = CBData.loadData('data/candles_2021-01-01T0:00:00__2021-01-31T18:00:00')
#import ipdb; ipdb.set_trace()
print(canCrit)
canCrit = canCrit.append(CBData.loadData('data/candles_2021-02-01T0:00:00__2021-02-28T18:00:00'))
print(canCrit)

hyperParam = {}
for g in np.linspace(0., 5.e-3, 51):
    print(g)
    _, h = benedict_bornder_constants(g=g, critical = True)

    hyperParam[str(g)] = {'g': g, 'h':h}
    #import ipdb; ipdb.set_trace()
    f = GHFilter(x = canCrit.iloc[0]['close'], dx = 0., 
                  dt = 1., g = hyperParam[str(g)]['g'], h = hyperParam[str(g)]['h'])
    results = f.batch_filter(list(canCrit['close']))
    #import ipdb; ipdb.set_trace()
    canCrit[f'g:{g:.4f}_x'] = results[1:,0]
    canCrit[f'g:{g:.4f}_dx'] = results[1:,1]
canCrit.set_index('time')

           time       low      high      open     close     volume
0    1612116000  32200.00  32328.42  32319.48  32207.16  55.639607
1    1612115940  32217.68  32344.88  32217.68  32319.48  19.588375
2    1612115880  32215.00  32415.77  32400.08  32223.85  94.305981
3    1612115820  32280.00  32487.46  32291.58  32400.48  43.410869
4    1612115760  32252.71  32382.77  32382.77  32291.58  84.612600
..          ...       ...       ...       ...       ...        ...
175  1609459500  28983.79  29031.71  29029.75  28997.34  14.545323
176  1609459440  29028.52  29052.35  29038.45  29034.30  20.515934
177  1609459380  29027.71  29059.40  29051.13  29041.72   8.934692
178  1609459320  29027.14  29079.56  29079.56  29053.76  32.481299
179  1609459260  29026.97  29083.75  29030.81  29079.56  27.489035

[44263 rows x 6 columns]
          time       low      high      open     close     volume
0   1612116000  32200.00  32328.42  32319.48  32207.16  55.639607
1   1612115940  32217.68  32344.88  32

/home/dockimble/miniconda3/envs/ghk/lib/python3.9/site-packages/filterpy/gh/gh_filter.py:1078: RuntimeWarning: invalid value encountered in double_scalars
  return (g, 0.8 * (2. - g_sqr - 2*(1-g_sqr)**.5) / g_sqr)


0.0004
0.0005
0.0006000000000000001
0.0007
0.0008
0.0009000000000000001
0.001
0.0011
0.0012000000000000001
0.0013000000000000002
0.0014
0.0015
0.0016
0.0017000000000000001
0.0018000000000000002
0.0019
0.002
0.0021000000000000003
0.0022
0.0023
0.0024000000000000002
0.0025
0.0026000000000000003
0.0027
0.0028
0.0029000000000000002
0.003
0.0031000000000000003
0.0032
0.0033
0.0034000000000000002
0.0035
0.0036000000000000003
0.0037
0.0038
0.0039000000000000003
0.004
0.0041
0.004200000000000001
0.0043
0.0044
0.0045000000000000005
0.0046
0.0047
0.0048000000000000004
0.0049
0.005


,low,high,open,close,volume,g:0.0000_x,g:0.0000_dx,g:0.0001_x,g:0.0001_dx,g:0.0002_x,...,g:0.0046_x,g:0.0046_dx,g:0.0047_x,g:0.0047_dx,g:0.0048_x,g:0.0048_dx,g:0.0049_x,g:0.0049_dx,g:0.0050_x,g:0.0050_dx
time,,,,,,,,,,,,,,,,,,,,,
1612116000,32200.00,32328.42,32319.48,32207.16,55.639607,32207.16,NaN,32207.160000,0.000000,32207.160000,...,32207.160000,0.000000,32207.160000,0.000000,32207.160000,0.000000,32207.160000,0.000000,32207.160000,0.000000
1612115940,32217.68,32344.88,32217.68,32319.48,19.588375,NaN,NaN,32207.171232,0.000002,32207.182464,...,32207.676672,0.000475,32207.687904,0.000496,32207.699136,0.000518,32207.710368,0.000539,32207.721600,0.000562
1612115880,32215.00,32415.77,32400.08,32223.85,94.305981,NaN,NaN,32207.172902,0.000002,32207.185799,...,32207.751542,0.000544,32207.764360,0.000568,32207.777175,0.000592,32207.789989,0.000617,32207.802801,0.000642
1612115820,32280.00,32487.46,32291.58,32400.48,43.410869,NaN,NaN,32207.192235,0.000006,32207.224458,...,32208.638635,0.001359,32208.670688,0.001419,32208.702738,0.001480,32208.734784,0.001542,32208.766826,0.001606
1612115760,32252.71,32382.77,32382.77,32291.58,84.612600,NaN,NaN,32207.200679,0.000007,32207.241332,...,32209.021518,0.001710,32209.061774,0.001785,32209.102022,0.001862,32209.142260,0.001940,32209.182489,0.002020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1612137900,32783.37,32893.03,32893.03,32783.73,48.219563,NaN,NaN,33978.464520,-1.416835,36522.693360,...,33208.025195,-0.763721,33203.521123,-0.770250,33198.811108,-0.777400,33193.912884,-0.785158,33188.843076,-0.793510
1612137840,32825.75,32911.98,32910.50,32893.03,40.682893,NaN,NaN,33976.939283,-1.416854,36521.654912,...,33205.816009,-0.765051,33201.295184,-0.771619,33196.569690,-0.778806,33191.657247,-0.786599,33186.574468,-0.794985
1612137780,32910.50,32977.88,32929.19,32910.51,31.938369,NaN,NaN,33975.415927,-1.416873,36520.620135,...,33203.696070,-0.766298,33199.160502,-0.772900,33194.421536,-0.780120,33189.496881,-0.787945,33184.403136,-0.796361


In [146]:
import matplotlib.pyplot as plt
%matplotlib widget
plt.clf()

days = np.linspace(0, len(canCritT['close'])/(24*60), len(canCritT['close']), endpoint=True)
canCritT = canCritT.sort_index()
canCritT['close'].plot(x = days, legend = True)


#plt.show()
for g in [0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]:
    print(f'g:{g:.4f}_x')
    canCritT[f'g:{g:.4f}_x'].plot(x = days, legend = True)

g = 0.0015
canCritT[f'g:{g:.4f}_x'].plot(x = days, legend = True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'canCritT' is not defined

In [ ]:

g = 0.0005
print(benedict_bornder_constants(g=g, critical = True))
print(benedict_bornder_constants(g=g, critical = False))
canCritT[f'g:{g:.4f}_x'].plot(x = days, legend = True)